## Import required libraries and set default values

In [1]:
import xlsxwriter
import numpy as np
import pandas as pd
import datetime as dt
from companies_house.api import CompaniesHouseAPI
date = dt.date.today()

## Request API access

In [2]:
##  if issues renew here https://developer.companieshouse.gov.uk/

#Insert your key here
API_key=r'YOURKEYHERE'
ch1=CompaniesHouseAPI(API_key)


## Input file is Excel or xls compatible file with column labelled "Company_numbers"

## Location of excel file containing Companies House company numbers

In [3]:
#Insert your file location here
file=r'YOUR FILE LOCATION'


In [4]:
Company_list= pd.read_excel(file)

In [38]:
Company_list['Company_numbers'] = Company_list[Company_list.columns[2]]


## Use an excel or csv file to provide a set of company numbers

In [18]:
Cleaned_company_df = Company_list.dropna()
Clean_company_list = Cleaned_company_df['Company_numbers'].tolist()

## Request info for all portfolio companies

In [19]:
Company_info=[]
for company in Clean_company_list:
    portfolio_information = ch1.get_company(company)
    Company_info.append(portfolio_information)

## Clean the list and isolate desired fields

In [20]:
Company_info_cleaned = list(filter(None, Company_info)) 

## Create overdue accounts list

In [21]:
df = pd.DataFrame(Company_info_cleaned)

In [22]:
overdue_true_or_false=[]
for i in range(len(Company_info_cleaned)):
    try:
        overdue_true_or_false_answer = Company_info_cleaned[i]['accounts']['overdue']
    except:
        overdue_true_or_false_answer = Company_info_cleaned[i]['company_status']
    overdue_true_or_false.append(overdue_true_or_false_answer)    

In [23]:
df['overdue_accounts']=pd.Series(overdue_true_or_false, index=df.index)

## Trim and colour the rows

In [24]:
Details_df=df[['company_name','company_status','date_of_cessation','company_number','overdue_accounts',]]

## Limited to 20 here to reduce info and print out but comment for normal use

In [39]:
Details_df =Details_df[0:20]
Details_df

,company_name,company_status,date_of_cessation,company_number,overdue_accounts
0,&SISTERS LTD,active,NaN,10510951,NaN
1,AFFRESOL LIMITED,liquidation,NaN,06441554,True
2,ANGELBERRY LTD,active,NaN,07806300,True
3,ARTIS FS LTD,active,NaN,10314551,NaN
4,"ASTERLEY BROS, LONDON LTD",active,NaN,09295823,NaN
5,ATLANTIC SEAWEEDS LIMITED,active,NaN,08340701,NaN
6,CANDIDATE ID LTD,active,NaN,SC516384,NaN
7,CELL GUIDANCE SYSTEMS LIMITED,active,NaN,07122793,NaN
10,CHIP FINANCIAL LTD,active,NaN,10113174,NaN
11,COMPARE AND SHARE LIMITED,dissolved,2016-10-04,08365213,True


In [26]:
Details_df['overdue_accounts']=Details_df['overdue_accounts'].apply(lambda x:'True' if x else np.nan)

## Use Groupby to eliminate any duplicates from follow on rounds

In [40]:
Details_df.groupby('company_status').count()

,company_name,date_of_cessation,company_number,overdue_accounts
company_status,,,,
active,11,0,11,1
dissolved,4,4,4,4
liquidation,2,0,2,2


## Create alphabetical company list and use traffic light colours to indicate status

In [28]:
Details_df = Details_df.sort_values(by=['company_name'])
#Details_df =Details_df.set_index('company_name')
Details_df = Details_df.drop_duplicates()

In [29]:
def Company_status_traffic_lights(s):
    if s.company_status =="active":
        return ['background-color: green']*(len(Details_df.columns))
    elif s.company_status=="dissolved":
        return['background-color: red']*(len(Details_df.columns))
    else:
        return ['background-color: orange']*(len(Details_df.columns))

In [30]:
df_traffic_light= Details_df.style.apply(Company_status_traffic_lights,axis=1)



In [43]:
df_traffic_light

,company_name,company_status,date_of_cessation,company_number,overdue_accounts
0,&SISTERS LTD,active,nan,10510951,nan
1,AFFRESOL LIMITED,liquidation,nan,06441554,True
2,ANGELBERRY LTD,active,nan,07806300,True
3,ARTIS FS LTD,active,nan,10314551,nan
4,"ASTERLEY BROS, LONDON LTD",active,nan,09295823,nan
5,ATLANTIC SEAWEEDS LIMITED,active,nan,08340701,nan
6,CANDIDATE ID LTD,active,nan,SC516384,nan
7,CELL GUIDANCE SYSTEMS LIMITED,active,nan,07122793,nan
10,CHIP FINANCIAL LTD,active,nan,10113174,nan
11,COMPARE AND SHARE LIMITED,dissolved,2016-10-04,08365213,True


In [32]:
DF_portfolio_hit_ratio = Details_df.groupby(['company_status',]).count()

In [33]:
DF_portfolio_hit_ratio = DF_portfolio_hit_ratio.filter(['company_name','overdue_accounts']).reset_index()

In [34]:
Dead_companies = Details_df.dropna(subset=['date_of_cessation']).sort_values(by=['date_of_cessation'])

In [44]:
Dead_companies.drop(['overdue_accounts'],axis=1)

,company_name,company_status,date_of_cessation,company_number
11,COMPARE AND SHARE LIMITED,dissolved,2016-10-04,08365213
12,CRUMPET CASHMERE LIMITED,dissolved,2017-05-07,08277744
14,DINE IN LIMITED,dissolved,2018-01-03,07458196
16,EAST END MANUFACTURING LIMITED,dissolved,2019-04-23,08190374


## Save output as an excel

In [46]:
#Set your output drive here

path = r'Path to store your portfolio'.format(date)


# Turn DFs to excel sheets and sort format

dfs = {'Hit_Ratio':DF_portfolio_hit_ratio,'Dead company timeline':Dead_companies}
writer = pd.ExcelWriter(path, engine='xlsxwriter')
for sheetname, df in dfs.items():  # loop through `dict` of dataframes
    df.to_excel(writer, sheet_name=sheetname,index=False)  # send df to writer
    worksheet = writer.sheets[sheetname]  # pull worksheet object
    for idx, col in enumerate(df):  # loop through all columns
        series = df[col]
        max_len = max((
            series.astype(str).map(len).max(),  # len of largest item
            len(str(series.name))  # len of column name/header
            )) + 10  # adding a little extra space
        fmt = writer.book.add_format({'align': 'center', 'valign': 'vcenter', 'text_wrap': True})
        worksheet.set_column(idx, idx, max_len,fmt)  

# Turn Traffic Light styler object to excel 

#format1 = workbook.add_format({'bg_color': '#FFC7CE','font_color': '#9C0006'})
# Note columns not auto-widened 
        
df_traffic_light.to_excel(writer,sheet_name='colour_coded',index=False)

writer.save()

writer.close()

## Citing companies_house_portfolio_assessment_tool

If you have found this useful please cite as below.

<blockquote> {companies_house_portfolio_assessment_tool,<br/>author = {Dai},<br/> title = {Companies_House_Portfolio_API},<br/>year = {2020},<br/>publisher = {GitHub},<br/>journal = {GitHub repository},<br/>howpublished = (https://github.com/Daidictates/Companies_house_portfolio),
}</blockquote>

# Acknowledgements

#### Thanks to all the library makers we've leant on